In [113]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [114]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IRP

In [115]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [116]:
import requests
import json

def get_balance(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8434R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "AFHR_FLPR_YN": "N",  # 시간외단일가여부
        "OFL_YN": "",  # 공란
        "INQR_DVSN": "01",  # 조회구분
        "UNPR_DVSN": "01",  # 단가구분
        "FUND_STTL_ICLD_YN": "N",  # 펀드결제분포함여부
        "FNCG_AMT_AUTO_RDPT_YN": "N",  # 융자금액자동상환여부        
        "PRCS_DVSN": "00",  # 처리구분(00: 전일매매포함)
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()
    

In [117]:
def get_balance_irp(access_token, app_key, app_secret, acc_no):
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/pension/inquire-balance"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC2208R"
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "ACCA_DVSN_CD":"00",
        "INQR_DVSN":"00",
        "CTX_AREA_FK100": "",  # 연속조회검색조건
        "CTX_AREA_NK100": ""  # 연속조회키
    }
    res = requests.get(url, headers=headers, params=params)
    return res.json()

In [118]:
json_0 = get_balance(token_0, api_key_0, api_secret_0, acc_no_0)
json_1 = get_balance(token_1, api_key_1, api_secret_1, acc_no_1)
json_2 = get_balance(token_2, api_key_2, api_secret_2, acc_no_2)
json_3 = get_balance(token_3, api_key_isa, api_secret_isa, acc_no_isa)
json_4 = get_balance_irp(token_4, api_key_irp, api_secret_irp, acc_no_irp)
json_4

{'ctx_area_fk100': '64012548^29^00^00^                                                                                  ',
 'ctx_area_nk100': '                                                                                                    ',
 'output1': [{'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 차이나CSI300',
   'pdno': '283580',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '20',
   'ord_psbl_qty': '20',
   'pchs_avg_pric': '10117.5000',
   'pchs_amt': '202350',
   'prpr': '10480',
   'evlu_amt': '209600',
   'evlu_pfls_amt': '7250',
   'evlu_erng_rt': '3.58290091'},
  {'cblc_dvsn_name': '사용자',
   'prdt_name': 'KODEX 200미국채혼합',
   'pdno': '284430',
   'item_dvsn_name': '현금',
   'thdt_buyqty': '0',
   'thdt_sll_qty': '0',
   'hldg_qty': '38',
   'ord_psbl_qty': '38',
   'pchs_avg_pric': '12590.2895',
   'pchs_amt': '478431',
   'prpr': '13340',
   'evlu_amt': '506920',
   'evlu_pfls_amt': '28489',
   'evlu_erng_rt': '5.95467267'},
  {

In [119]:
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

def get_balance_json(json):
    cash = json['output2'][0]['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_pfls_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)
        
        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [120]:
import pandas as pd


def get_balance_json_irp(json):
    cash = json['output2']['prvs_rcdl_excc_amt']
    ap = pd.DataFrame.from_records(json['output1'])
    if(ap.size > 0):
        ap = ap[["pdno", "prdt_name", "evlu_amt", "prpr", "hldg_qty", "pchs_amt", "pchs_avg_pric", "evlu_erng_rt"]]
        ap.columns = ["코드", "이름", "평가금액", "단가", "수량", "매입금액", "매입단가", "수익률"]
        new_row = {"코드": "0", "이름": "현금", "평가금액": cash, "단가": 1, "수량": cash, "매입금액": cash, "매입단가": 1, "수익률": 0}
        new_row_df = pd.DataFrame([new_row])

        # concat을 사용하여 새로운 행 추가
        ap = pd.concat([ap, new_row_df], ignore_index=True)

        # 데이터 타입 변환
        ap["평가금액"] = pd.to_numeric(ap["평가금액"], errors='coerce')
        ap["매입금액"] = pd.to_numeric(ap["매입금액"], errors='coerce')
        ap["단가"] = pd.to_numeric(ap["단가"], errors='coerce')
        ap["매입단가"] = pd.to_numeric(ap["매입단가"], errors='coerce')
        ap["수량"] = pd.to_numeric(ap["수량"], errors='coerce')
        ap["수익률"] = pd.to_numeric(ap["수익률"], errors='coerce')
        ap["계좌"] = json['ctx_area_fk100'][:8] + '-' + json['ctx_area_fk100'][9:11]
    return ap

In [121]:
df_0 = get_balance_json(json_0)
df_1 = get_balance_json(json_1)
df_2 = get_balance_json(json_2)
df_3 = get_balance_json(json_3)
df_4 = get_balance_json_irp(json_4)
df = pd.concat([df_0, df_1, df_2, df_3, df_4], ignore_index=True)
# df_0 = df_0.sort_values(by='이름')
df_1 = df_1.sort_values(by='이름')
df_2 = df_2.sort_values(by='이름')
df_3 = df_3.sort_values(by='이름')
df_4 = df_4.sort_values(by='이름')

total_1 = df_1["평가금액"].sum()
total_2 = df_2["평가금액"].sum()
total_3 = df_3["평가금액"].sum()
total_4 = df_4["평가금액"].sum()

In [122]:
df_1["비중"] = df_1["평가금액"] / total_1 * 100
df_1

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌,비중
2,411060,ACE KRX금현물,6990570,15230,459,6802840,14821.00,2.75,64267316-22,14.16
10,476750,ACE 미국30년국채엔화노출액티브(H),1044765,9585,109,982813,9016.63,6.30,64267316-22,2.12
0,360200,ACE 미국S&P500,5794875,19125,303,5330680,17593.00,8.70,64267316-22,11.74
6,456880,ACE 미국달러SOFR금리(합성),3010500,11150,270,2945731,10910.11,2.19,64267316-22,6.10
11,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2519700,11100,227,2698366,11887.07,-6.62,64267316-22,5.10
8,468380,KODEX iShares미국하이일드액티브,1435200,11040,130,1420250,10925.00,1.05,64267316-22,2.91
7,464540,RISE 25-03 회사채(AA-이상)액티브,2396060,50980,47,2394495,50946.70,0.06,64267316-22,4.85
9,473330,SOL 미국30년국채커버드콜(합성),10140130,10130,1001,9694000,9684.32,4.60,64267316-22,20.54
3,446720,SOL 미국배당다우존스,3364500,11215,300,3136694,10455.65,7.26,64267316-22,6.81
5,452360,SOL 미국배당다우존스(H),2278000,11390,200,2138084,10690.42,6.54,64267316-22,4.61


In [123]:
total_1

49370034

In [124]:
df_2

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,1401160,15230,92,1330258,14459.33,5.32,64012548-22
1,360200,ACE 미국S&P500,1453500,19125,76,1338800,17615.79,8.56,64012548-22
7,456880,ACE 미국달러SOFR금리(합성),144950,11150,13,140018,10770.62,3.52,64012548-22
2,390390,KODEX 미국반도체MV,484785,23085,21,434120,20672.38,11.67,64012548-22
9,483230,RISE 25-06 은행채(AA+이상)액티브,1860545,50285,37,1858510,50230.00,0.10,64012548-22
8,469050,RISE 미국반도체NYSE(H),540000,15000,36,477917,13275.47,12.99,64012548-22
6,452360,SOL 미국배당다우존스(H),2164100,11390,190,1998428,10518.04,8.29,64012548-22
3,396500,TIGER Fn반도체TOP10,447480,12430,36,469560,13043.33,-4.70,64012548-22
5,449170,TIGER KOFR금리액티브(합성),3712275,106065,35,3703648,105818.51,0.23,64012548-22
0,329750,TIGER 미국달러단기채권액티브,1049750,12350,85,1013038,11918.09,3.62,64012548-22


In [125]:
total_2

13270245

In [126]:
df_3

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
1,411060,ACE KRX금현물,1964670,15230,129,1939575,15035.47,1.29,64043278-01
0,114800,KODEX 인버스,1039125,4075,255,1010475,3962.65,2.83,64043278-01
3,473330,SOL 미국30년국채커버드콜(합성),1215600,10130,120,1217010,10141.75,-0.11,64043278-01
2,449170,TIGER KOFR금리액티브(합성),3075885,106065,29,3071965,105929.83,0.12,64043278-01
4,0,현금,1904,1,1904,1904,1.00,0.00,64043278-01


In [127]:
total_3

7297184

In [128]:
df_4

,코드,이름,평가금액,단가,수량,매입금액,매입단가,수익률,계좌
4,411060,ACE KRX금현물,4858370,15230,319,4785871,15002.73,1.51,64012548-29
3,365780,ACE 국고채10년,273225,91075,3,261365,87121.67,4.54,64012548-29
2,360200,ACE 미국S&P500,631125,19125,33,527245,15977.12,19.70,64012548-29
6,456880,ACE 미국달러SOFR금리(합성),1449500,11150,130,1441550,11088.85,0.55,64012548-29
1,284430,KODEX 200미국채혼합,506920,13340,38,478431,12590.29,5.95,64012548-29
8,468380,KODEX iShares미국하이일드액티브,1214400,11040,110,1201700,10924.55,1.06,64012548-29
0,283580,KODEX 차이나CSI300,209600,10480,20,202350,10117.50,3.58,64012548-29
7,464540,RISE 25-03 회사채(AA-이상)액티브,8513660,50980,167,8507035,50940.33,0.08,64012548-29
9,473330,SOL 미국30년국채커버드콜(합성),3039000,10130,300,2984859,9949.53,1.81,64012548-29
5,449170,TIGER KOFR금리액티브(합성),13152060,106065,124,13110565,105730.36,0.32,64012548-29


In [129]:
total_4

33934732

In [130]:
# 그룹화하여 조건에 맞게 계산
result = df.groupby('코드').agg({
    '이름': 'first',
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
    '매입단가': 'mean',
}).reset_index()

result["비중"] = result["평가금액"] / result["평가금액"].sum() * 100
result["수익률"] = (result["평가금액"] - result["매입금액"]) / result["매입금액"] * 100
result.dropna(inplace=True)

# 결과 출력
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률
0,0,현금,120282,1.00,120282,120282,1.00,0.12,0.00
1,114800,KODEX 인버스,1039125,4075.00,255,1010475,3962.65,0.99,2.84
2,283580,KODEX 차이나CSI300,209600,10480.00,20,202350,10117.50,0.20,3.58
3,284430,KODEX 200미국채혼합,506920,13340.00,38,478431,12590.29,0.48,5.95
4,329750,TIGER 미국달러단기채권액티브,1049750,12350.00,85,1013038,11918.09,1.00,3.62
5,360200,ACE 미국S&P500,7879500,19125.00,412,7196725,17061.97,7.53,9.49
6,365780,ACE 국고채10년,273225,91075.00,3,261365,87121.67,0.26,4.54
7,390390,KODEX 미국반도체MV,484785,23085.00,21,434120,20672.38,0.46,11.67
8,396500,TIGER Fn반도체TOP10,2212540,12430.00,178,2395175,13302.00,2.12,-7.63
9,411060,ACE KRX금현물,15580290,15230.00,1023,15222184,14894.04,14.90,2.35


In [131]:
eval_total = result['평가금액'].sum()
eval_total

104574152

In [132]:
# 카테고리 분류
# 114800	KODEX 인버스
# 283580	KODEX 차이나CSI300
# 284430	KODEX 200미국채혼합
# 329750	TIGER 미국달러단기채권액티브

원화현금 = ["0", "449170",]
달러현금 = ["456880",]
국내주식 = ["396500", "114800"]
미국주식 = ["360200", "390390", "402970", "469050", "480020", "482730", "446720", "452360"]
해외주식 = ["283580", ]
미국채권 = ["284430", "329750", "473330", "476750"]
국내채권 = ["365780", "464540", "483230"]
하이일드 = ["468380", ]
대채투자 = ["411060", ]

result['카테고리'] = '기타'
result.loc[result['코드'].isin(원화현금), '카테고리'] = '원화현금'
result.loc[result['코드'].isin(달러현금), '카테고리'] = '달러현금'
result.loc[result['코드'].isin(국내주식), '카테고리'] = '국내주식'
result.loc[result['코드'].isin(미국주식), '카테고리'] = '미국주식'
result.loc[result['코드'].isin(해외주식), '카테고리'] = '해외주식'
result.loc[result['코드'].isin(미국채권), '카테고리'] = '미국채권'
result.loc[result['코드'].isin(국내채권), '카테고리'] = '국내채권'
result.loc[result['코드'].isin(하이일드), '카테고리'] = '하이일드'
result.loc[result['코드'].isin(대채투자), '카테고리'] = '대채투자'
result.sort_values(by='카테고리', inplace=True)
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리
1,114800,KODEX 인버스,1039125,4075.00,255,1010475,3962.65,0.99,2.84,국내주식
8,396500,TIGER Fn반도체TOP10,2212540,12430.00,178,2395175,13302.00,2.12,-7.63,국내주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1860545,50285.00,37,1858510,50230.00,1.78,0.11,국내채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10909720,50980.00,214,10901530,50943.52,10.43,0.08,국내채권
6,365780,ACE 국고채10년,273225,91075.00,3,261365,87121.67,0.26,4.54,국내채권
13,456880,ACE 미국달러SOFR금리(합성),4604950,11150.00,413,4527299,10923.19,4.40,1.72,달러현금
9,411060,ACE KRX금현물,15580290,15230.00,1023,15222184,14894.04,14.90,2.35,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2519700,11100.00,227,2698366,11887.07,2.41,-6.62,미국주식
16,469050,RISE 미국반도체NYSE(H),540000,15000.00,36,477917,13275.47,0.52,12.99,미국주식
12,452360,SOL 미국배당다우존스(H),4442100,11390.00,390,4136512,10604.23,4.25,7.39,미국주식


In [133]:
# 새로운 '분류' 컬럼 추가
result['대분류'] = result['카테고리']
result.loc[result['카테고리'].str.contains('현금'), '대분류'] = '현금'
result.loc[result['카테고리'].str.contains('주식'), '대분류'] = '주식'
result.loc[result['카테고리'].str.contains('채권'), '대분류'] = '채권'
result.loc[result['카테고리'].str.contains('하이일드'), '대분류'] = '채권'
result

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,1039125,4075.00,255,1010475,3962.65,0.99,2.84,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2212540,12430.00,178,2395175,13302.00,2.12,-7.63,국내주식,주식
21,483230,RISE 25-06 은행채(AA+이상)액티브,1860545,50285.00,37,1858510,50230.00,1.78,0.11,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10909720,50980.00,214,10901530,50943.52,10.43,0.08,국내채권,채권
6,365780,ACE 국고채10년,273225,91075.00,3,261365,87121.67,0.26,4.54,국내채권,채권
13,456880,ACE 미국달러SOFR금리(합성),4604950,11150.00,413,4527299,10923.19,4.40,1.72,달러현금,현금
9,411060,ACE KRX금현물,15580290,15230.00,1023,15222184,14894.04,14.90,2.35,대채투자,대채투자
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2519700,11100.00,227,2698366,11887.07,2.41,-6.62,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),540000,15000.00,36,477917,13275.47,0.52,12.99,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4442100,11390.00,390,4136512,10604.23,4.25,7.39,미국주식,주식


In [134]:
# 카테고리 별 비중 계산
대분류 = result.groupby('대분류').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = 대분류['평가금액'].sum()
대분류['비중'] = 대분류['평가금액'] / total_amount * 100
대분류["수익률"] = (대분류["평가금액"] - 대분류["매입금액"]) / 대분류["매입금액"] * 100
대분류

,대분류,평가금액,단가,수량,매입금액,비중,수익률
0,대채투자,15580290,15230.00,1023,15222184,14.90,2.35
1,주식,27608685,12829.50,2312,26668984,26.40,3.52
2,채권,32689255,31098.12,2147,32013506,31.26,2.11
3,현금,28695922,39072.00,120921,28556671,27.44,0.49


In [135]:
# 카테고리 별 비중 계산
category_group = result.groupby('카테고리').agg({
    '평가금액': 'sum',
    '단가': 'mean',
    '수량': 'sum',
    '매입금액': 'sum',
}).reset_index()

total_amount = category_group['평가금액'].sum()
category_group['비중'] = category_group['평가금액'] / total_amount * 100
category_group["수익률"] = (category_group["평가금액"] - category_group["매입금액"]) / category_group["매입금액"] * 100
category_group

,카테고리,평가금액,단가,수량,매입금액,비중,수익률
0,국내주식,3251665,8252.50,433,3405650,3.11,-4.52
1,국내채권,13043490,64113.33,254,13021405,12.47,0.17
2,달러현금,4604950,11150.00,413,4527299,4.40,1.72
3,대채투자,15580290,15230.00,1023,15222184,14.90,2.35
4,미국주식,24147420,14472.86,1859,23060984,23.09,4.71
5,미국채권,16996165,11351.25,1653,16370151,16.25,3.82
6,원화현금,24090972,53033.00,120508,24029372,23.04,0.26
7,하이일드,2649600,11040.00,240,2621950,2.53,1.05
8,해외주식,209600,10480.00,20,202350,0.20,3.58


In [136]:
# '대분류'가 '채권'인 항목 필터링
bond_df = result[result['대분류'] == '채권']

# 총 평가금액 계산
total_bond_amount = bond_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
bond_df.loc[:, '비중'] = bond_df['평가금액'] / total_bond_amount * 100
bond_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
21,483230,RISE 25-06 은행채(AA+이상)액티브,1860545,50285.00,37,1858510,50230.00,5.69,0.11,국내채권,채권
14,464540,RISE 25-03 회사채(AA-이상)액티브,10909720,50980.00,214,10901530,50943.52,33.37,0.08,국내채권,채권
6,365780,ACE 국고채10년,273225,91075.00,3,261365,87121.67,0.84,4.54,국내채권,채권
4,329750,TIGER 미국달러단기채권액티브,1049750,12350.00,85,1013038,11918.09,3.21,3.62,미국채권,채권
3,284430,KODEX 200미국채혼합,506920,13340.00,38,478431,12590.29,1.55,5.95,미국채권,채권
17,473330,SOL 미국30년국채커버드콜(합성),14394730,10130.00,1421,13895869,9925.20,44.04,3.59,미국채권,채권
18,476750,ACE 미국30년국채엔화노출액티브(H),1044765,9585.00,109,982813,9016.63,3.20,6.30,미국채권,채권
15,468380,KODEX iShares미국하이일드액티브,2649600,11040.00,240,2621950,10924.77,8.11,1.05,하이일드,채권


In [137]:
# '대분류'가 '주식'인 항목 필터링
stock_df = result[result['대분류'] == '주식']

# 총 평가금액 계산
total_stock_amount = stock_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
stock_df.loc[:, '비중'] = stock_df['평가금액'] / total_stock_amount * 100
stock_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
1,114800,KODEX 인버스,1039125,4075.00,255,1010475,3962.65,3.76,2.84,국내주식,주식
8,396500,TIGER Fn반도체TOP10,2212540,12430.00,178,2395175,13302.00,8.01,-7.63,국내주식,주식
19,480020,ACE 미국빅테크7+ 15%프리미엄분배(합성),2519700,11100.00,227,2698366,11887.07,9.13,-6.62,미국주식,주식
16,469050,RISE 미국반도체NYSE(H),540000,15000.00,36,477917,13275.47,1.96,12.99,미국주식,주식
12,452360,SOL 미국배당다우존스(H),4442100,11390.00,390,4136512,10604.23,16.09,7.39,미국주식,주식
10,446720,SOL 미국배당다우존스,3364500,11215.00,300,3136694,10455.65,12.19,7.26,미국주식,주식
7,390390,KODEX 미국반도체MV,484785,23085.00,21,434120,20672.38,1.76,11.67,미국주식,주식
5,360200,ACE 미국S&P500,7879500,19125.00,412,7196725,17061.97,28.54,9.49,미국주식,주식
20,482730,TIGER 미국S&P500+10%프리미엄초단기,4916835,10395.00,473,4980650,10529.92,17.81,-1.28,미국주식,주식
2,283580,KODEX 차이나CSI300,209600,10480.00,20,202350,10117.50,0.76,3.58,해외주식,주식


In [138]:
# '대분류'가 '현금'인 항목 필터링
cash_df = result[result['대분류'] == '현금']

# 총 평가금액 계산
total_cash_amount = cash_df['평가금액'].sum()

# 각각의 항목에 대해 비중 계산
cash_df.loc[:, '비중'] = cash_df['평가금액'] / total_cash_amount * 100
cash_df

,코드,이름,평가금액,단가,수량,매입금액,매입단가,비중,수익률,카테고리,대분류
13,456880,ACE 미국달러SOFR금리(합성),4604950,11150.00,413,4527299,10923.19,16.05,1.72,달러현금,현금
11,449170,TIGER KOFR금리액티브(합성),23970690,106065.00,226,23909090,105872.15,83.53,0.26,원화현금,현금
0,0,현금,120282,1.00,120282,120282,1.00,0.42,0.00,원화현금,현금



## 최근 영업일

In [139]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.07.25


In [140]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20240725
